In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load the data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

#train_df = train_df.dropna()

# Preprocessing: fill NaNs and get text and target columns
# train_df['text'] = train_df['text'].fillna('')
# test_df['text'] = test_df['text'].fillna('')
# X = train_df['text']
# y = train_df['target']
train_df.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [79]:
# train_df['keyword'].unique()
# we're going to drop the location and id because I dont really think its necessary

train_df = train_df.drop(columns=["id", "location"])

# Drop any rows with missing values in the 'text' or 'target' columns
train_df = train_df.dropna(subset=['text', 'target'])

In [80]:
# stole this from https://www.kaggle.com/code/kodaifukuda0311/disaster-tweets-classification-with-rnn-and-cnn
# Preprocessing
import re
import string
# Remove URL from the text
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

pattern = re.compile(r'https?://(\S+|www)\.\S+')
for t in train_df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r'', t))
    if len(matches) > 0:
        break

#Remove puctuations
def remove_punct(text):
    translator = str.maketrans('', '', string.punctuation) # Create a translation table to delete all punctuation
    return text.translate(translator) # Use this translation table to transform the text by removing punctuation characters

#pattern = re.compile(r'https?://(\S+|www)\.\S+')
for t in train_df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r'', t))
    if len(matches) > 0:
        break

# Apply the functions
train_df['text'] = train_df.text.map(remove_URL) # map(lambda x : remove_URL(x))
train_df['text'] = train_df.text.map(remove_punct)

@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
t
@bbcmtd Wholesale Markets ablaze 
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
t
@bbcmtd Wholesale Markets ablaze 


In [81]:
#count the number of unique words

def count_words(sent):
    unique_words = set()
    for s in sent.values:
        for word in s.split():
            unique_words.add(word)
    return len(unique_words)

In [82]:
unique_words = count_words(train_df.text)
print(unique_words)

17203


In [83]:
# Use 'text' as input (X) and 'target' as output (y)
X = train_df['text']
y = train_df['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

In [84]:
# Tokenize the text
#tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer = Tokenizer(num_words=unique_words)
tokenizer.fit_on_texts(X_train)

x_seq = tokenizer.texts_to_sequences(X_train)
val_seq = tokenizer.texts_to_sequences(X_val)

X_pad = pad_sequences(x_seq, padding='post', maxlen=100)


print(X_train[1:3])
print(x_seq[1:3])

[' Had a personalinjury accident this summer Read our advice amp see how a solicitor can help OtleyHour'
 'Sometimes logic gets drowned out in emotion but its gotta surface at some point']
[[95, 2, 4485, 112, 18, 395, 161, 102, 4486, 33, 84, 63, 2, 2938, 73, 152, 4487], [1155, 2939, 598, 555, 36, 3, 4488, 26, 42, 1787, 2210, 16, 77, 955]]
